# Example usage of segmentation to btrack to napari visualization

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [ ]:
import glob
import os

import btrack
import napari

import numpy as np

from skimage.io import imread

In [ ]:
PATH = "/Users/dane/Documents/CellTrackingProjects/AU565/images/AU02001/Analysis/PC/intermediate_files/"

In [ ]:
files = glob.glob(os.path.join(PATH, 'tracking', 'A1','field_1', 'masks','mask*.tif'))

# sort the files numerically
#files = sorted(files, key=lambda f: int(f[len(os.path.join(PATH, 'labels_')):-4]))
files = sorted(files)
phase_stack_filename = glob.glob(os.path.join(PATH, "AU02001_P_A1_1_reg_stack.tif"))[0]
phase_stack = imread(phase_stack_filename)

In [ ]:
files
#phase_stack_filename

### method 1 - using a numpy array

In this example, each image from the timelapse is a 3D volume (32 x 1200 x 1200) and there are 10 timepoints

In [ ]:
def segmentation_arr(files):
    """Segmentation as numpy array."""
    
    stack = []
    for filename in files:
        img = imread(filename)
        stack.append(img)
    return np.stack(stack, axis=0)

In [ ]:
stack = segmentation_arr(files)

Now we print out the shape of the stack (T, Z, Y, X):

In [ ]:
stack.shape

### method 2 - using a generator

This is useful if you're resource constrained and don't want to load all of the image data, or they are stored in an unusual format. Note that the generator produces a numpy array for each image...

In [ ]:
def segmentation_generator(files):
    """Segmentation generator"""
    
    for filename in files:
        img = imread(filename)
        yield img

In [ ]:
generator = segmentation_generator(files)

## localizing the objects

Now we use a utility function to localise the objects in the segmentation, and also apply anisotropic scaling (using the `scale` option, here the z-values are scaled by 2x). Note that we can also use scikit-image `regionprops` to calculate properties for each object, using the `properties` keyword:

obj_from_arr = btrack.utils.segmentation_to_objects(stack, properties=('area', ))

# inspect the first object
obj_from_arr[0]

In [ ]:
obj_from_generator = btrack.utils.segmentation_to_objects(
    generator, 
    properties = ('area', 'major_axis_length')
)

In [ ]:
# inspect the first object
obj_from_generator[0]

## run btrack with the objects

We will use the objects from the generator here.

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('../models/cell_config_AU565.json')
    tracker.max_search_radius = 100

    # append the objects to be tracked
    tracker.append(obj_from_generator)

    # set the volume
    tracker.volume=((0, 1408), (0, 1040), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=200)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(PATH, 'tracking.h5'), obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=2)
    
    #tracks = tracker.tracks

## visualize with napari

Note that we also set the scale of the images here to account for the anisotropy.

In [ ]:
#viewer = napari.Viewer()
#viewer = napari.view_image(img_p_stack, name = "phase image")

viewer = napari.view_image(phase_stack, name = "phase image")
viewer.add_labels(stack,  name='Segmentation')
viewer.add_tracks(data, properties=properties, graph=graph, name='Tracks')

napari.run()

In [ ]:
graph

In [ ]:
graph